In [4]:
#!pip install torch
#!pip install torchvision

     |████████████████████████████████| 292 kB 804 kB/s            
     |████████████████████████████████| 25.9 MB 10.6 MB/s            


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torchvision.io import read_image
from torchvision.transforms import ToTensor

1) Выбрать задачу по классификации изображений, загрузить данные,
провести анализ (распределение классов, особенности данных и т.д.) -
2 балла

Мною была выбрана задача классификации птиц: https: .kaggle.com/competitions/sp23-bird-classif-cation-osu-ai-cl-ub/

In [2]:
 # Для проверки будет удобно записать в переменную path свой path - код будет работать
#path = '/media/egor/Windows/Users/Egor/Documents/Files/Учёба/ML_and_DL_Digital_department/3_Lab/'
path = 'C:/Users/Egor/Documents/Files/Учёба/ML_and_DL_Digital_department/3_Lab/'
df = pd.read_csv(f'{path}birds.csv')
df

,image_id,class_id,filepaths,labels,scientific_label,data_set
0,0,0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
1,1,0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
2,2,0,train/ABBOTTS BABBLER/003.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
3,3,0,train/ABBOTTS BABBLER/004.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
4,4,0,train/ABBOTTS BABBLER/005.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
...,...,...,...,...,...,...
72871,72871,449,valid/YELLOW HEADED BLACKBIRD/1.jpg,YELLOW HEADED BLACKBIRD,Xanthocephalus,valid
72872,72872,449,valid/YELLOW HEADED BLACKBIRD/2.jpg,YELLOW HEADED BLACKBIRD,Xanthocephalus,valid
72873,72873,449,valid/YELLOW HEADED BLACKBIRD/3.jpg,YELLOW HEADED BLACKBIRD,Xanthocephalus,valid
72874,72874,449,valid/YELLOW HEADED BLACKBIRD/4.jpg,YELLOW HEADED BLACKBIRD,Xanthocephalus,valid


In [3]:
df['class_id'] = df['class_id'].apply(str)
df_train = df[df['data_set'] == 'train']
df_valid = df[df['data_set'] == 'valid']
print(df['class_id'].describe())
print(df_train['class_id'].describe())
print(df_valid['class_id'].describe())

count     72876
unique      450
top         262
freq        253
Name: class_id, dtype: object
count     70626
unique      450
top         262
freq        248
Name: class_id, dtype: object
count     2250
unique     450
top          0
freq         5
Name: class_id, dtype: object


In [4]:
print(df['labels'].describe())
print(df_train['labels'].describe())
print(df_valid['labels'].describe())
# Видим, что class_id и labels совпадают - с датасетом всё хорошо.

count           72876
unique            450
top       HOUSE FINCH
freq              253
Name: labels, dtype: object
count           70626
unique            450
top       HOUSE FINCH
freq              248
Name: labels, dtype: object
count                2250
unique                450
top       MANGROVE CUCKOO
freq                    5
Name: labels, dtype: object


In [5]:
df['scientific_label'].describe()
# У нас 447 видов птиц (по видимому есть несколько морф, которые относятся к одному виду),
# самый распостранённый класс Александров кольчатый попугай (зелёненький такой)
# У него 354 изображения, что составляет ~0,5%,
# что в ~2 раза больше, чем среднее количество фотографий 72676/447 = 163 на класс.
# Тем не менее, т.к. у нас очень много классов, то можно считать датасет сбалансированным по классам.
# Т.к. превышение 1 класса из 447 в 2 раза выше среднего - не является чем-то критичным.

count                   72876
unique                    447
top       Psittacula eupatria
freq                      354
Name: scientific_label, dtype: object

In [6]:
df['data_set'].describe()
# Видим, что 97% набора являются тренирововчным, и только 3 отданы на валидацию.

count     72876
unique        2
top       train
freq      70626
Name: data_set, dtype: object

In [7]:
df_test = pd.read_csv(f'{path}sample_solution.csv')
df_test

,image_id,class_id
0,72876,427
1,72877,188
2,72878,142
3,72879,413
4,72880,371
...,...,...
2245,75121,167
2246,75122,373
2247,75123,277
2248,75124,126


In [8]:
df_test['class_id'] = df_test['class_id'].apply(str)
df_test['class_id'].describe()
# В папке test находится ~3% от общего количества изображений
# Из 450 классов тренировочного набора присутствует 445 классов,
# т.е. 5 классов не предствалены в тестовом наборе

count     2250
unique     445
top        305
freq        12
Name: class_id, dtype: object

In [4]:
df['class_id'] = df['class_id'].apply(int)
df_train = df[df['data_set'] == 'train']
df_valid = df[df['data_set'] == 'valid']

2) Написать класс для датасета, наследуемый от PyTorch Dataset, в
котором реализовать init, len, getitem методы - 3 балла

In [5]:
class Birds(Dataset):
    
    def __init__(self, path, annotation, transform = None, target_transform = None):
        self.img_labels = annotation['image_id']
        self.img_class = annotation['class_id']
        self.data_dir = path
        self.img_dir = annotation['filepaths']
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = str(self.data_dir + self.img_dir[idx])
        image = read_image(img_path)
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

3) Поместить датасет в dataloader, разбив на батчи - 3 балла

In [8]:
train_data = Birds(path, df_train)
train_dataloader = DataLoader(train_data, batch_size=100, shuffle=True, drop_last=True)
valid_data = Birds(path, df_valid)
valid_dataloader = DataLoader(valid_data, shuffle=False, drop_last=True)

In [9]:
# Выбор одного случайного изображения
for img, label in train_dataloader:
    break
print(img.shape)
print(img[0].shape)
# Переворачиваем порядок измерений из (channels, height, width) в (height, width, channels)
img = img[0].permute(1, 2, 0)
print(type(img))
# Преобразуем тензор в массив numpy
img_np = img.numpy()
img = (img * 255).byte()
print(type(img_np))
# Создаем изображение с помощью PIL
img_pil = Image.fromarray(img_np)
print(type(img_pil))

#img_pil.show()
# Изначальный размер изображения batch*3*224*224

torch.Size([100, 3, 224, 224])
torch.Size([3, 224, 224])
<class 'torch.Tensor'>
<class 'numpy.ndarray'>
<class 'PIL.Image.Image'>


4) Обучить модель, состоящую из полносвязных слоёв или обучить модель,
состоящую из сверточных и полносвязных слоев (например, Conv -
Pooling - Conv - Linear - Linear) - 5 баллов

Решил попробовать немного другую архитектуру: Conv - Norm - Conv - Pooling - Conv - Linear - Linear

In [10]:
class MyFirstArchiectureNN(nn.Module):
    # Изначально мой батч размерности batch*3*224*224
    def __init__(self):
        super(MyFirstArchiectureNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=1)
        # Теперь он свёрнут до batch*6*222*222
        self.pool1 = nn.MaxPool2d(3, stride=2)
        # Pooling уменьшил размерность в 2 раза batch*6*110*110
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=10, stride=1, padding=1)
        # Теперь он свёрнут до batch*12*103*103
        self.pool2 = nn.MaxPool2d(2, stride=3)
        # Теперь он свёрнут до batch*12*51*51
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=20, stride=1, padding=1)
        # Теперь он свёрнут до batch*24*34*34
        self.pool3 = nn.MaxPool2d(2, stride=3)
        
        self.linear1 = nn.Linear(24*6*6, 512)
        # Теперь свёрнут до 1*
        self.linear2 = nn.Linear(512, 450)
        #Сворачиваем до 450 (сколько целевых классов)

    def forward(self, input):
        output = F.relu(self.conv1(input))
        print(output.shape)
        output = self.pool1(output)
        print(output.shape)
        output = F.relu(self.conv2(output))
        print(output.shape)
        output = self.pool2(output)
        print(output.shape)
        output = F.relu(self.conv3(output))
        print(output.shape)
        output = self.pool3(output)
        print(output.shape)
        output = output.view(output.size(0), 24*6*6)
        output = F.relu(self.linear1(output))
        print(output.shape)
        output = self.linear2(output)
        print(output.shape)
        return output

In [11]:
device_count = torch.cuda.device_count()
print(f"Доступно устройств GPU: {device_count}")

for i in range(device_count):
    device_name = torch.cuda.get_device_name(i)
    print(f"Устройство {i}: {device_name}")

Доступно устройств GPU: 0


In [12]:

# Определяем устройство для обучения
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [13]:
# Создаём объекс нашего собственного класса нейросети и закидываем на видеокарту
my_first_nn = MyFirstArchiectureNN().to(device)
# Выбираем функцию потерь (вроде как CrossEntropy хороша для задач классификации)
criterion = nn.CrossEntropyLoss().to(device)
# Выбираем оптимизатор Adam, т.к. современный и активно используется
optimizer = torch.optim.Adam(my_first_nn.parameters(), lr=0.001)

train_losses = []
valid_losses = []
num_epochs = 3 # Для пробы взял немного эпох

#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb=5'
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

for epoch in range(num_epochs):
    for batch, labels in train_dataloader:
        #print(batch.shape)
        #print(labels.shape)
        # Закидываем наши данные на видеокарту
        batch, labels = batch.to(device), labels.to(device)
        my_first_nn.train()
        batch = batch.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = my_first_nn(batch.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss = {loss.item():.4f}')
    
    my_first_nn.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for batch, labels in valid_dataloader:
            outputs = my_first_nn(batch)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    print(f'Valid loss = {loss.item():.4f}')
    
    train_losses.append(train_loss / len(train_dataloader))
    valid_losses.append(valid_loss / len(valid_dataloader))
    
print('Training is finished')

torch.save(my_first_nn.state_dict(), 'my_first_bird_classifier.pth')


torch.Size([100, 6, 222, 222])
torch.Size([100, 6, 110, 110])
torch.Size([100, 12, 103, 103])
torch.Size([100, 12, 34, 34])
torch.Size([100, 24, 17, 17])
torch.Size([100, 24, 6, 6])
torch.Size([100, 512])
torch.Size([100, 450])


IndexError: Target 12564 is out of bounds.

In [ ]:
# Вывод массивов значений потерь
print(f'Training Losses: {losses}')
print(f'Validation Losses: {valid_losses}')

In [ ]:
my_model = MyFirstArchiectureNN()
my_model.load_state_dict(torch.load('my_first_bird_classifier.pth'))

5) Обучить или дообучить одну из моделей, включенных в torchvision
например, ResNet (модели можно посмотреть тут
https: pytorch.org/vision/stable/models.html) - 4 балла

6) Дать комментарии по результатам 3-5 пунктов, отрисовать графики -
3 балла